In [ ]:
# 인코딩 잘못된 파일 수정(2021년 2월, 3월)
import os
import pandas as pd

# ===============================================================
# 1) 컬럼명이 "깨진 한글(euc-kr -> latin1 잘못 디코딩)"인지 판단
# ===============================================================
def looks_broken_korean_header(columns):
    text = "".join(columns)

    # 이미 한글이 정상 포함되어 있으면 깨지지 않았다고 판단
    if any("가" <= ch <= "힣" for ch in text):
        return False

    # 전형적인 euc-kr mojibake 발생 문자들
    mojibake_chars = set("´ÀÁÂÃÄÅÆÇÈÉÊËÌÍÎÏÐÑÒÓÔÕÖ×ØÙÚÛÜÝÞß¿¼½¾¸µ¹²³°")
    if any(ch in mojibake_chars for ch in text):
        return True

    return False


# ===============================================================
# 2) 한 개 CSV 파일 복구하기
# ===============================================================
def fix_one_csv(path, dry_run=False):
    print(f"\n[검사] {path}")

    # 1단계 — cp949 / euc-kr / utf-8-sig 로 읽어서 정상 여부 체크
    for enc in ["cp949", "euc-kr", "utf-8-sig"]:
        try:
            df = pd.read_csv(path, encoding=enc, low_memory=False, nrows=5)
            # 헤더에 한글이 있으면 정상 파일로 판단
            if not looks_broken_korean_header(df.columns.tolist()):
                print(f"  → {enc} 로 정상 읽힘 (깨짐 아님) → 스킵")
                return False
        except Exception:
            continue

    # 2단계 — latin1 로 읽어서 헤더 모양 확인
    try:
        df_full = pd.read_csv(path, encoding="latin1", low_memory=False)
    except Exception as e:
        print(f"  ⚠ latin1 로도 못 읽음 → 스킵: {e}")
        return False

    if not looks_broken_korean_header(df_full.columns.tolist()):
        print("  → latin1 로 읽혔지만 깨진 패턴은 아님 → 스킵")
        return False

    print("  ✅ 깨진 CSV로 판단 → 복원 시작 (latin1 → cp949)")

    # 3단계 — 컬럼명 복구
    new_cols = []
    for c in df_full.columns:
        try:
            new_c = c.encode("latin1").decode("cp949")
        except Exception:
            new_c = c
        new_cols.append(new_c)
    df_full.columns = new_cols

    # 4단계 — 문자열 셀 값들도 복구
    for col in df_full.columns:
        if df_full[col].dtype == object:
            try:
                df_full[col] = df_full[col].apply(
                    lambda x: x.encode("latin1").decode("cp949")
                    if isinstance(x, str) else x
                )
            except Exception:
                # 일부 값에서만 실패해도 전체는 계속 가게
                pass

    # 5단계 — 복원된 파일 저장
    out_path = path.replace(".csv", "_CLEANED.csv")
    if not dry_run:
        df_full.to_csv(out_path, index=False, encoding="utf-8-sig")
    print(f"  💾 복원된 파일 저장 완료 → {out_path}")

    return True


# ===============================================================
# 3) 연도별 하위 폴더까지 전부 돌며 깨진 CSV 자동 복원
# ===============================================================
def fix_all_broken_csvs(base_dir, dry_run=False):
    if not os.path.exists(base_dir):
        print(f"❌ 폴더가 존재하지 않습니다: {base_dir}")
        return []

    broken_files = []

    # os.walk 를 쓰면 2021, 2022 같은 연도 폴더도 자동으로 다 탐색됨
    for current_dir, _, files in os.walk(base_dir):
        for fname in files:
            # 따릉이 시간대별 파일만 대상으로 하고 싶으면 조건 추가
            if not fname.lower().endswith(".csv"):
                continue
            if "공공자전거 이용정보(시간대별)" not in fname:
                continue

            path = os.path.join(current_dir, fname)
            fixed = fix_one_csv(path, dry_run=dry_run)
            if fixed:
                broken_files.append(path)

    print("\n====================== 결과 요약 ======================")
    if broken_files:
        print("복원된(또는 깨진 것으로 탐지된) CSV:")
        for b in broken_files:
            print(" -", b)
    else:
        print("깨진 CSV가 발견되지 않았습니다.")
    print("=======================================================")

    return broken_files


# ===============================================================
# ⚡ 실행: 여기 경로만 당신 컴퓨터 기준으로 맞추면 끝
#    (연도별로 2021, 2022 폴더가 그 안에 있는 '최상위' 폴더)
# ===============================================================
base_dir = r"D:\0516_컴퓨터\공공자전거\서울특별시 공공자전거 이용정보(시간대별)"

fix_all_broken_csvs(base_dir, dry_run=False)



[검사] D:\0516_컴퓨터\공공자전거\서울특별시 공공자전거 이용정보(시간대별)\2017\서울특별시 공공자전거 이용정보(시간대별)_17.01.csv
  → utf-8-sig 로 정상 읽힘 (깨짐 아님) → 스킵

[검사] D:\0516_컴퓨터\공공자전거\서울특별시 공공자전거 이용정보(시간대별)\2017\서울특별시 공공자전거 이용정보(시간대별)_17.02.csv
  → utf-8-sig 로 정상 읽힘 (깨짐 아님) → 스킵

[검사] D:\0516_컴퓨터\공공자전거\서울특별시 공공자전거 이용정보(시간대별)\2017\서울특별시 공공자전거 이용정보(시간대별)_17.03.csv
  → utf-8-sig 로 정상 읽힘 (깨짐 아님) → 스킵

[검사] D:\0516_컴퓨터\공공자전거\서울특별시 공공자전거 이용정보(시간대별)\2017\서울특별시 공공자전거 이용정보(시간대별)_17.04.csv
  → utf-8-sig 로 정상 읽힘 (깨짐 아님) → 스킵

[검사] D:\0516_컴퓨터\공공자전거\서울특별시 공공자전거 이용정보(시간대별)\2017\서울특별시 공공자전거 이용정보(시간대별)_17.05.csv
  → utf-8-sig 로 정상 읽힘 (깨짐 아님) → 스킵

[검사] D:\0516_컴퓨터\공공자전거\서울특별시 공공자전거 이용정보(시간대별)\2017\서울특별시 공공자전거 이용정보(시간대별)_17.06.csv
  → cp949 로 정상 읽힘 (깨짐 아님) → 스킵

[검사] D:\0516_컴퓨터\공공자전거\서울특별시 공공자전거 이용정보(시간대별)\2017\서울특별시 공공자전거 이용정보(시간대별)_17.07.csv
  → utf-8-sig 로 정상 읽힘 (깨짐 아님) → 스킵

[검사] D:\0516_컴퓨터\공공자전거\서울특별시 공공자전거 이용정보(시간대별)\2017\서울특별시 공공자전거 이용정보(시간대별)_17.08.csv
  → cp949 로 정상 읽힘 (깨짐 아님) → 스킵

[검사] D:\0516_컴퓨터\공공자전거\서울특별시 공공자전거 이용정보(시간대별)\2

['D:\\0516_컴퓨터\\공공자전거\\서울특별시 공공자전거 이용정보(시간대별)\\2021\\서울특별시 공공자전거 이용정보(시간대별)_21.02.csv',
 'D:\\0516_컴퓨터\\공공자전거\\서울특별시 공공자전거 이용정보(시간대별)\\2021\\서울특별시 공공자전거 이용정보(시간대별)_21.03.csv']

In [ ]:
#파일 중 워크시트>csv로 변경
import os
import pandas as pd

def convert_excel_to_csv(base_dir):
    print("\n================ EXCEL → CSV 변환 시작 ================")
    converted = []

    # 하위 폴더까지 모두 탐색
    for current_dir, _, files in os.walk(base_dir):
        for fname in files:
            ext = fname.lower().split(".")[-1]
            if ext not in ["xls", "xlsx"]:
                continue

            path = os.path.join(current_dir, fname)
            out_path = path.rsplit(".", 1)[0] + "_CONVERTED.csv"

            print(f"\n[변환] {path}")
            try:
                df = pd.read_excel(path, engine="openpyxl")
                df.to_csv(out_path, index=False, encoding="utf-8-sig")
                print(f"  → 저장 완료: {out_path}")
                converted.append(out_path)
            except Exception as e:
                print(f"  ⚠ 변환 실패: {e}")

    print("\n================ 결과 요약 ================")
    if converted:
        print("다음 파일들이 CSV로 변환되었습니다:")
        for c in converted:
            print(" -", c)
    else:
        print("변환할 엑셀 파일이 없었습니다.")
    print("==================================================")

    return converted

# 실행 (네 PC 경로 그대로 사용)
base_dir = r"D:\0516_컴퓨터\공공자전거\서울특별시 공공자전거 이용정보(시간대별)"
convert_excel_to_csv(base_dir)



================ EXCEL → CSV 변환 시작 ================

[변환] D:\0516_컴퓨터\공공자전거\서울특별시 공공자전거 이용정보(시간대별)\2018\서울특별시 공공자전거 이용정보(시간대별)_18.01.xlsx
  → 저장 완료: D:\0516_컴퓨터\공공자전거\서울특별시 공공자전거 이용정보(시간대별)\2018\서울특별시 공공자전거 이용정보(시간대별)_18.01_CONVERTED.csv

[변환] D:\0516_컴퓨터\공공자전거\서울특별시 공공자전거 이용정보(시간대별)\2018\서울특별시 공공자전거 이용정보(시간대별)_18.02.xlsx
  → 저장 완료: D:\0516_컴퓨터\공공자전거\서울특별시 공공자전거 이용정보(시간대별)\2018\서울특별시 공공자전거 이용정보(시간대별)_18.02_CONVERTED.csv

[변환] D:\0516_컴퓨터\공공자전거\서울특별시 공공자전거 이용정보(시간대별)\2018\서울특별시 공공자전거 이용정보(시간대별)_18.03.xlsx
  → 저장 완료: D:\0516_컴퓨터\공공자전거\서울특별시 공공자전거 이용정보(시간대별)\2018\서울특별시 공공자전거 이용정보(시간대별)_18.03_CONVERTED.csv

[변환] D:\0516_컴퓨터\공공자전거\서울특별시 공공자전거 이용정보(시간대별)\2018\서울특별시 공공자전거 이용정보(시간대별)_18.04.xlsx
  → 저장 완료: D:\0516_컴퓨터\공공자전거\서울특별시 공공자전거 이용정보(시간대별)\2018\서울특별시 공공자전거 이용정보(시간대별)_18.04_CONVERTED.csv

[변환] D:\0516_컴퓨터\공공자전거\서울특별시 공공자전거 이용정보(시간대별)\2018\서울특별시 공공자전거 이용정보(시간대별)_18.05.xlsx
  → 저장 완료: D:\0516_컴퓨터\공공자전거\서울특별시 공공자전거 이용정보(시간대별)\2018\서울특별시 공공자전거 이용정보(시간대별)_18.05_CONVERTED.csv

[변환] D:\0516_컴퓨터

['D:\\0516_컴퓨터\\공공자전거\\서울특별시 공공자전거 이용정보(시간대별)\\2018\\서울특별시 공공자전거 이용정보(시간대별)_18.01_CONVERTED.csv',
 'D:\\0516_컴퓨터\\공공자전거\\서울특별시 공공자전거 이용정보(시간대별)\\2018\\서울특별시 공공자전거 이용정보(시간대별)_18.02_CONVERTED.csv',
 'D:\\0516_컴퓨터\\공공자전거\\서울특별시 공공자전거 이용정보(시간대별)\\2018\\서울특별시 공공자전거 이용정보(시간대별)_18.03_CONVERTED.csv',
 'D:\\0516_컴퓨터\\공공자전거\\서울특별시 공공자전거 이용정보(시간대별)\\2018\\서울특별시 공공자전거 이용정보(시간대별)_18.04_CONVERTED.csv',
 'D:\\0516_컴퓨터\\공공자전거\\서울특별시 공공자전거 이용정보(시간대별)\\2018\\서울특별시 공공자전거 이용정보(시간대별)_18.05_CONVERTED.csv',
 'D:\\0516_컴퓨터\\공공자전거\\서울특별시 공공자전거 이용정보(시간대별)\\2018\\서울특별시 공공자전거 이용정보(시간대별)_18.06_CONVERTED.csv',
 'D:\\0516_컴퓨터\\공공자전거\\서울특별시 공공자전거 이용정보(시간대별)\\2018\\서울특별시 공공자전거 이용정보(시간대별)_18.07_CONVERTED.csv',
 'D:\\0516_컴퓨터\\공공자전거\\서울특별시 공공자전거 이용정보(시간대별)\\2018\\서울특별시 공공자전거 이용정보(시간대별)_18.08_CONVERTED.csv',
 'D:\\0516_컴퓨터\\공공자전거\\서울특별시 공공자전거 이용정보(시간대별)\\2018\\서울특별시 공공자전거 이용정보(시간대별)_18.09_CONVERTED.csv',
 'D:\\0516_컴퓨터\\공공자전거\\서울특별시 공공자전거 이용정보(시간대별)\\2018\\서울특별시 공공자전거 이용정보(시간대별)_18.10_CONVERTED.csv',
 'D:\\0516_컴퓨터\\공공자전

In [17]:
####
import pandas as pd
import os
import glob
import gc  # 메모리 회수용

# ============================================================
# 0. CSV / XLSX 자동 판별해서 읽는 함수
# ============================================================
def read_any_file(path):
    ext = os.path.splitext(path)[1].lower()

    # 1) 엑셀 파일(.xlsx, .xls)
    if ext in [".xlsx", ".xls"]:
        df = pd.read_excel(path, engine="openpyxl")
        print(f"    → Excel로 읽음: {os.path.basename(path)}")
        return df

    # 2) CSV 파일(.csv): 인코딩 여러 개 시도
    if ext == ".csv":
        encodings = ["utf-8-sig", "cp949", "euc-kr", "ISO-8859-1"]
        last_err = None
        for enc in encodings:
            try:
                df = pd.read_csv(path, encoding=enc, low_memory=False)
                print(f"    → CSV({enc})로 읽음: {os.path.basename(path)}")
                return df
            except Exception as e:
                last_err = e
        # 그래도 안 되면, 에러 무시하고 적당히 깨지더라도 읽기
        try:
            df = pd.read_csv(path, encoding="utf-8", encoding_errors="ignore", low_memory=False)
            print(f"    → CSV(utf-8, errors='ignore')로 읽음: {os.path.basename(path)}")
            return df
        except Exception as e:
            raise ValueError(f"CSV 파일을 읽을 수 없습니다: {path} | 마지막 에러: {last_err} / {e}")

    # 3) 그 외 확장자
    raise ValueError(f"지원하지 않는 파일 타입입니다: {path}")


# ============================================================
# 1. 처리할 연도 목록
#    👉 메모리 부족하면 예: years = [2021] 만 두고 돌린 후,
#       커널 재시작하고 2022, 2023 ... 이런 식으로 나눠 돌려도 됨.
# ============================================================
years = [2025]

# 2. 따릉이 기본 폴더
bike_base_dir = r"D:\0516_컴퓨터\공공자전거\서울특별시 공공자전거 이용정보(시간대별)"

# 3. 결과 저장 폴더
out_dir = r"C:\Users\user\Documents\code\Time-series-demand-forecasting-team2_backup\data\interim\01.bike\02.bike_weather"
os.makedirs(out_dir, exist_ok=True)

# ============================================================
# 🔥 연도별 전체 병합 루프
# ============================================================
for year in years:
    print(f"\n===== {year}년 처리 시작 =====")

    # --------------------------------------------------------
    # 1) 날씨 데이터 읽기 + 날짜/시간 컬럼 정제
    # --------------------------------------------------------
    weather_url = (
        "https://raw.githubusercontent.com/"
        "JJINwj/Time-series-demand-forecasting-team2/main/"
        f"data/interim/01.bike/01.weather/modi_weather_{year}.csv"
    )

    weather = None
    for enc in ["cp949", "euc-kr", "utf-8-sig", "ISO-8859-1"]:
        try:
            weather = pd.read_csv(weather_url, encoding=enc, low_memory=False)
            print(f"  → Weather 파일 {enc} 로 읽힘")
            break
        except Exception as e:
            last_err = e
            continue

    if weather is None:
        print(f"  ⚠ 날씨 데이터 읽기 실패: {weather_url} | 마지막 에러: {last_err}")
        continue

    weather_cols = weather.columns.tolist()

    # 날짜 컬럼 자동 찾기
    date_candidates = [c for c in weather_cols if ("날짜" in c) or ("date" in c.lower()) or ("일자" in c)]
    if not date_candidates:
        raise ValueError(f"날씨 파일에서 날짜 컬럼을 찾지 못했습니다: {weather_cols}")
    weather.rename(columns={date_candidates[0]: "날짜"}, inplace=True)

    # 시간 컬럼 자동 찾기
    # ⚠ '시정(10m)' 같은 건 걸러야 해서 조건을 조금 타이트하게
    time_candidates = []
    for c in weather_cols:
        cl = c.lower()
        if "시간" in c:
            time_candidates.append(c)
        elif cl in ["hour", "hh", "h"]:
            time_candidates.append(c)
        elif c.strip().endswith("시"):
            time_candidates.append(c)

    if not time_candidates:
        raise ValueError(f"날씨 파일에서 시간 컬럼을 찾지 못했습니다: {weather_cols}")
    weather.rename(columns={time_candidates[0]: "시간"}, inplace=True)

    # 시간값 정제 → '0', '00', '0시~1시' 등 모두 2자리 문자열로
    weather["시간"] = (
        weather["시간"]
        .astype(str)
        .str.extract(r"(\d+)")[0]   # 숫자만 추출
        .astype(int)
        .astype(str)
        .str.zfill(2)
    )

    # 불필요한 컬럼 제거 (있다면)
    drop_cols = [c for c in weather.columns if c.startswith("Unnamed")]
    if drop_cols:
        weather = weather.drop(columns=drop_cols)

    print("  → Weather 컬럼:", weather.columns.tolist())

    # --------------------------------------------------------
    # 2) 따릉이 연도 폴더 및 파일 목록
    # --------------------------------------------------------
    bike_dir = os.path.join(bike_base_dir, str(year))
    print("  📁 따릉이 폴더:", bike_dir)

    yy = str(year)[2:4]

    pattern_csv = os.path.join(bike_dir, f"서울특별시 공공자전거 이용정보(시간대별)_{yy}*.csv")
    pattern_xls = os.path.join(bike_dir, f"서울특별시 공공자전거 이용정보(시간대별)_{yy}*.xls")
    pattern_xlsx = os.path.join(bike_dir, f"서울특별시 공공자전거 이용정보(시간대별)_{yy}*.xlsx")

    bike_files = sorted(
        glob.glob(pattern_csv) +
        glob.glob(pattern_xls) +
        glob.glob(pattern_xlsx)
    )

    print("  따릉이 파일 목록:")
    for f in bike_files:
        print("   -", os.path.basename(f))

    if not bike_files:
        print("  ⚠ 이 연도에는 파일이 없습니다. 스킵합니다.")
        continue

    # --------------------------------------------------------
    # 3) 연도별 결과 저장 파일 설정
    # --------------------------------------------------------
    save_path = os.path.join(out_dir, f"bike_weather_{year}.csv")
    if os.path.exists(save_path):
        os.remove(save_path)

    first_chunk = True

    # =====================================================
    # 4) 월별 병합 (메모리 절약)
    # =====================================================
    for path in bike_files:
        fname = os.path.basename(path)
        print(f"\n  ▶ 처리 중: {fname}")

        # 4-1. 파일 읽기
        try:
            df_bike = read_any_file(path)
        except Exception as e:
            print(f"    ⚠ 이 파일은 읽지 못했습니다. 스킵: {fname} | 에러: {e}")
            continue

        # 4-2. 헤더의 작은따옴표 제거
        df_bike.columns = [col.strip().strip("'") for col in df_bike.columns]

        # 4-2.5 정말 필요한 컬럼만 남기기 (메모리 절약)
        # 👉 실제 데이터 컬럼명에 맞게 리스트는 필요하면 조정
        bike_keep_cols = [
            "대여일자", "대여시간",      # 날짜/시간
            "대여일시",                 # 통합일시가 있으면 여기서 분리할 예정
            "대여소번호", "대여소", 
            "대여구분코드", "성별", "연령대코드",
            "이용거리", "이용시간", "이용건수"
        ]
        existing_cols = [c for c in bike_keep_cols if c in df_bike.columns]
        if existing_cols:
            df_bike = df_bike[existing_cols].copy()
        else:
            # 혹시 keep 리스트에 없는 이상한 스키마면, 전체 유지
            df_bike = df_bike.copy()

        # 4-3. 텍스트 컬럼 작은따옴표 제거
        clean_cols = ["대여일자", "대여시간", "대여소번호", "대여소",
                      "대여구분코드", "성별", "연령대코드"]
        for col in clean_cols:
            if col in df_bike.columns:
                df_bike[col] = (
                    df_bike[col]
                    .astype(str)
                    .str.strip()
                    .str.strip("'")
                )

        # 4-3.5 숫자 컬럼은 가볍게 형변환 (float32)
        numeric_cols = ["이용거리", "이용시간", "이용건수"]
        for col in numeric_cols:
            if col in df_bike.columns:
                df_bike[col] = pd.to_numeric(df_bike[col], errors="coerce").astype("float32")

        # ------------------------------------------------
        # 4-4. '대여일자' / '대여시간' 자동 추론 및 생성
        # ------------------------------------------------
        cols = list(df_bike.columns)

        # (1) '대여일자'가 없고 '대여일시' 같은 통합 컬럼이 있으면 분리
        if "대여일자" not in df_bike.columns:
            datetime_candidates = [c for c in cols if "일시" in c and "대여" in c]
            if datetime_candidates:
                cand = datetime_candidates[0]
                print(f"    ↪ '{cand}'에서 '대여일자'/'대여시간' 분리합니다.")
                dt = pd.to_datetime(df_bike[cand], errors="coerce")
                df_bike["대여일자"] = dt.dt.strftime("%Y-%m-%d")
                df_bike["대여시간"] = dt.dt.hour.astype(str).str.zfill(2)
            else:
                # 다른 이름의 날짜 컬럼 탐색
                date_candidates = [c for c in cols if ("일자" in c or "날짜" in c) and ("반납" not in c)]
                if date_candidates:
                    cand = date_candidates[0]
                    print(f"    ↪ '{cand}' 컬럼을 '대여일자'로 사용합니다.")
                    df_bike.rename(columns={cand: "대여일자"}, inplace=True)

        # (2) 시간 컬럼 추론 ('대여시간', '대여시간대', '시간대' 등)
        if "대여시간" not in df_bike.columns:
            time_candidates = [c for c in cols if ("시간" in c or "시" in c) and ("반납" not in c)]
            if time_candidates:
                cand = time_candidates[0]
                print(f"    ↪ '{cand}' 컬럼을 '대여시간'으로 사용합니다.")
                df_bike["대여시간"] = df_bike[cand]

        # ------------------------------------------------
        # 4-5. 최종적으로 날짜/시간 둘 다 없으면 스킵
        # ------------------------------------------------
        if "대여일자" not in df_bike.columns or "대여시간" not in df_bike.columns:
            print("    ⚠ '대여일자' 또는 '대여시간' 컬럼이 없어 스킵:", fname)
            print("       - 실제 컬럼 목록:", df_bike.columns.tolist())
            del df_bike
            gc.collect()
            continue

        # ------------------------------------------------
        # 4-6. '날짜', '시간' 컬럼 생성
        # ------------------------------------------------
        df_bike["날짜"] = df_bike["대여일자"]
        df_bike["시간"] = (
            df_bike["대여시간"]
            .astype(str)
            .str.extract(r"(\d+)")[0]  # '0시~1시' → '0'
            .astype(int)
            .astype(str)
            .str.zfill(2)
        )

        print("    - 월 파일 shape:", df_bike.shape)

        # ------------------------------------------------
        # 4-7. 날씨와 병합
        # ------------------------------------------------
        merged_chunk = pd.merge(
            df_bike,
            weather,
            on=["날짜", "시간"],
            how="left"
        )
        print("    - 병합 후 shape:", merged_chunk.shape)

        # ------------------------------------------------
        # 4-8. CSV로 저장 (연도별 하나만)
        # ------------------------------------------------
        if first_chunk:
            merged_chunk.to_csv(save_path, index=False, encoding="utf-8-sig")
            first_chunk = False
        else:
            merged_chunk.to_csv(
                save_path,
                index=False,
                encoding="utf-8-sig",
                mode="a",
                header=False
            )

        # ------------------------------------------------
        # 4-9. 월 처리 끝 → 메모리 회수
        # ------------------------------------------------
        del df_bike
        del merged_chunk
        gc.collect()

    # 연도 처리 끝 → 날씨 메모리 회수
    del weather
    gc.collect()

    print(f"\n  ✅ {year}년 최종 병합 파일 저장 → {save_path}")
    print("  Exists:", os.path.exists(save_path))

print("\n===== 전체 연도 병합 루프 종료 =====")



===== 2025년 처리 시작 =====
  → Weather 파일 cp949 로 읽힘
  → Weather 컬럼: ['날짜', '시간', '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '시정(10m)', '적설(cm)']
  📁 따릉이 폴더: D:\0516_컴퓨터\공공자전거\서울특별시 공공자전거 이용정보(시간대별)\2025
  따릉이 파일 목록:
   - 서울특별시 공공자전거 이용정보(시간대별)_25.01.csv
   - 서울특별시 공공자전거 이용정보(시간대별)_25.02.csv
   - 서울특별시 공공자전거 이용정보(시간대별)_25.03.csv
   - 서울특별시 공공자전거 이용정보(시간대별)_25.04.csv
   - 서울특별시 공공자전거 이용정보(시간대별)_25.05.csv
   - 서울특별시 공공자전거 이용정보(시간대별)_25.06.csv

  ▶ 처리 중: 서울특별시 공공자전거 이용정보(시간대별)_25.01.csv
    → CSV(cp949)로 읽음: 서울특별시 공공자전거 이용정보(시간대별)_25.01.csv
    - 월 파일 shape: (1610875, 9)
    - 병합 후 shape: (1610875, 15)

  ▶ 처리 중: 서울특별시 공공자전거 이용정보(시간대별)_25.02.csv
    → CSV(cp949)로 읽음: 서울특별시 공공자전거 이용정보(시간대별)_25.02.csv
    - 월 파일 shape: (759111, 9)
    - 병합 후 shape: (759111, 15)

  ▶ 처리 중: 서울특별시 공공자전거 이용정보(시간대별)_25.03.csv
    → CSV(cp949)로 읽음: 서울특별시 공공자전거 이용정보(시간대별)_25.03.csv
    - 월 파일 shape: (2651685, 9)
    - 병합 후 shape: (2651685, 15)

  ▶ 처리 중: 서울특별시 공공자전거 이용정보(시간대별)_25.04.csv
    → CSV(cp949)로 읽음: 서울특별시 공공자